## Scraping the top repositories for topics of github

TODO (Intro):

Introduction about web scraping
Introduction about GitHub and the problem statement
Use of Python, requests, Beautiful Soup, Pandas

Here are the steps we'll follow:

We're going to scrape https://github.com/topics
We'll get a list of topics. For each topic, we'll get topic title, topic page URL and topic description
For each topic, we'll get the top 25 repositories in the topic from the topic page
For each repository, we'll grab the repo name, username, stars and repo URL
For each topic we'll create a CSV file in the following format:
Repo Name,Username,Stars,Repo URL
three.js,mrdoob,69700,https://github.com/mrdoob/three.js
libgdx,libgdx,18300,https://github.com/libgdx/libgdx

Scrape the list of topics from Github
Explain how you'll do it.

use requests to downlaod the page
user BS4 to parse and extract information
convert to a Pandas dataframe
Let's write a function to download the page.



In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def get_topic_page():
    #download the page
    topic_url='https://github.com/topics'
    response=requests.get(topic_url)
    #check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page{}'.format(topic_url))
    # parse using BeautifulSoup
    topic_doc=BeautifulSoup(response.text,"html.parser")
    return topic_doc


In [2]:
doc = get_topic_page()

Let's create some helper functions to parse information from the page.

To get topic titles, we can pick p tags:


In [3]:
def get_topic_titles(doc):
    selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
    topic_title_tags = doc.find_all('p', {'class': selection_class})
    topic_titles = []
    for tag in topic_title_tags:
        topic_titles.append(tag.text)
    return topic_titles

In [4]:
#get_topic_titles can be used to get the list of titles
titles = get_topic_titles(doc)
len(titles)
titles[:5]

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android']

# Similarly we have defined functions for descriptions and URLs.

In [6]:
def get_topic_descs(doc):
    discription_class={"class":"f5 color-fg-muted mb-0 mt-1"}
    topic_discription_class=soup.find_all('p',discription_class)
    topic_description=[]
    for tag in topic_discription_class:
        topic_description.append(tag.text.strip())
    return topic_description


In [7]:
def get_topic_urls(doc):
    topic_link_tags = soup.find_all('a',{'class': "no-underline flex-grow-0"})
    topic_urls=[]
    base_url="https://github.com"
    for tag in topic_link_tags:
        topic_urls.append(base_url + tag["href"])
    return topic_urls

## A single function for all this information

In [10]:
def scrape_topics():
    topic_url='https://github.com/topics'
    response=requests.get(topic_url)
    #check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page{}'.format(topic_url))
    # parse using BeautifulSoup
    topic_doc=BeautifulSoup(response.text,"html.parser")
    topic_dict={"title":get_topic_titles(topic_doc),"description":get_topic_descs(topic_doc),"url":get_topic_urls(topic_doc)}
    return pd.DataFrame(topic_dict)

## Get the top 25 Repositories from a topic page


In [16]:

def get_topic_page(topic_url):
    # Download the page
    response = requests.get(topic_url)
    # Check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    # Parse using Beautiful soup
    topic_doc = BeautifulSoup(response.text, 'html.parser')
    return topic_doc

In [17]:
def parse_star_strip(star_str):
    star_str=star_str.strip()
    if star_str[-1]=='k':
       return int(float(star_str[:-1])*1000)
    return int(star_str)

In [19]:
def get_repo_info(h3_tag,star_tag):  # gives all information about repository
    base_url="https://github.com"
    a_tag=h3_tag.find_all('a')
    user_name=a_tag[0].text.strip()
    repo_name=a_tag[1].text.strip()
    repo_url=base_url+a_tag[1]['href']
    stars=parse_star_strip(star_tag.text.strip())
    
    return user_name,repo_name,repo_url,stars

In [20]:
def get_topic_repos(topic_doc):
    #get the h3 tag containing repo title,repo url,username
    h3_selection_class="f3 color-fg-muted text-normal lh-condensed"
    repo_tag=topic_doc.find_all('h3',{'class':h3_selection_class})
    # get star tags
    star_tags=topic_doc.find_all('span',{'class':"Counter js-social-count"})
    topic_repo_dict={
    "user_name":[],
    "repo_name":[],
    "repo_url":[],
    "stars":[]
    }
    # get repo info
    for i in range(len(repo_tag)):
      repo_info = get_repo_info(repo_tag[i],star_tags[i])
      topic_repo_dict["user_name"].append(repo_info[0])
      topic_repo_dict["repo_name"].append(repo_info[1])
      topic_repo_dict["repo_url"].append(repo_info[2])
      topic_repo_dict["stars"].append(repo_info[3])
    return pd.DataFrame(topic_repo_dict)

In [21]:
def scrape_topic(topic_url,topic_name):
    fname=topic_name+'.csv'
    if os.path.exists(fname):
        print("The file {} already exists.Skipping...".format(fname))
        return
    topic_df=get_topic_repos(get_topic_page(topic_url))
    topic_df.to_csv(fname,index=None)

In [24]:
def get_topic_titles(soup):
    topic_title_tags=soup.find_all('p',{"class":"f3 lh-condensed mb-0 mt-1 Link--primary"})
    topic_titles=[]
    
    for tags in topic_title_tags:
        topic_titles.append(tags.text)
    return topic_titles

def get_topic_descs(soup):
     discription_class={"class":"f5 color-fg-muted mb-0 mt-1"}
     topic_discription_class=soup.find_all('p',discription_class)
     topic_description=[]
     for tag in topic_discription_class:
         topic_description.append(tag.text.strip())
     return topic_description

def get_topic_urls(soup):
    topic_link_tags = soup.find_all('a',{'class': "no-underline flex-grow-0"})
    topic_urls=[]
    base_url="https://github.com"
    for tag in topic_link_tags:
        topic_urls.append(base_url + tag["href"])
    return topic_urls
def scrape_topics():
    topics_url=("https://github.com/topics")
    response=requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to load page{}'.format(topic_url))
    topics_dict={
        'title':get_topic_titles(soup),
        'description':get_topic_descs(soup),
        'url':get_topic_urls(soup)
    }
    return pd.DataFrame(topics_dict)
   
def scrape_topics_repos():
    print("Sraping list of topics:")
    topics_df=scrape_topics()
    for index,row in topics_df.iterrows():
        print('scraping top repositories for"{}"'.format(row['title']))
        scrape_topic(row['url'],row['title'])
    

    

## References and Future work:
summary 
- From github get all topics from topic page
- then get all the top repos from a paerticular topic page
- information of top repos includes repository name , username, repository link, stars 

References

- from BeautifulSoup documentation
- youtube
- pandas documentation (for making datasets)


Future work

- will get all the information for trending repositories and will explore more repositories